# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [576]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2025-05-12 17:05:08,259 | INFO     |: Logger demo_notebook.log already initialized.
2025-05-12 17:05:08,260 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [579]:
# STOP_PLOTTER()
# Start the plotter
START_PLOTTER()

2025-05-12 17:05:41,570 | ERROR    |: Plotter is already running


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [682]:
import time
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        # packet should be in the form of "yaw_control?|target_yaw_value|max_clockwise_speed|min_clockwise_speed|max_counter_clockwise_speed|min_counter_clockwise_speed|P_GAIN|D_GAIN|alpha"
        control_timestamps = []
        yaw = []
        distance = []

        def extract_control_data(uuid, byte_array):
            string = byte_array.decode("utf-8") 
            split_str = string.split("|")
            control_timestamps.append(int(split_str[0]))
            yaw.append(float(split_str[1]))
            distance.append(int(split_str[2]))
    
        self.ble.start_notify(self.ble.uuid['RX_STRING'], extract_control_data)
        self.ble.send_command(CMD.LOCALIZE, "90|60|90|60|8|0.7|0.1")

        async def sleep_for_30_secs():
            await asyncio.sleep(30)

        await sleep_for_30_secs()

        self.ble.stop_notify(self.ble.uuid['RX_STRING'])

        np_meters = np.array(distance) / 1000.0
        return np_meters[np.newaxis].T, np.array([])



In [683]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-05-12 18:10:36,262 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:81:80:25:12:64
2025-05-12 18:10:41,698 | INFO     |: Connected to c0:81:80:25:12:64


In [672]:
# ble.send_command(CMD.GET_TOF1, "")
# distance1 = int(ble.receive_string(ble.uuid['RX_STRING']))
# print(f"Measured distance: {distance1}")


ble.send_command(CMD.GET_TOF2, "")
distance1 = int(ble.receive_string(ble.uuid['RX_STRING']))
print(f"Measured distance: {distance1}")

Exception: Not connected to a BLE device

In [573]:
# int targetDistances[7] = {600, 1600, 500, 500, 700, 1000, 1000};
targetDistances = [2000, 1600, 800, 1400, 1800, 1800, 2000]
# int targetAngles[6] = {270, 180, 270, 0, 90, 180};
targetAngles = [270, 180, 270, 0, 90, 180]


# ble.send_command(CMD.YAW_CONTROL, "1|90|250|80|250|80|8|0.7|0.1")

# packet in the form "openLoopNav?|target_yaw_value|max_clockwise_speed|min_clockwise_speed|max_counter_clockwise_speed|min_counter_clockwise_speed|P_GAIN|D_GAIN|alpha"
ble.send_command(
    CMD.OPEN_LOOP_NAV, 
    f"1|100|80|100|80|8|0.7|0.1|"
    f"{targetAngles[0]}|{targetAngles[1]}|{targetAngles[2]}|{targetAngles[3]}|{targetAngles[4]}|{targetAngles[5]}|50|"
    f"{targetDistances[0]}|{targetDistances[1]}|{targetDistances[2]}|{targetDistances[3]}|{targetDistances[4]}|{targetDistances[5]}|{targetDistances[6]}"
)

2025-05-12 16:10:09,305 | INFO     |: Disconnected from C0:81:80:25:12:64


In [560]:
ble.send_command(CMD.YAW_CONTROL, "0|270|250|80|250|80|8|0.7|0.1")


In [336]:
# TURN OFF
ble.send_command(
    CMD.OPEN_LOOP_NAV, 
    f"0|250|80|250|80|3|0.7|0.1|"
    f"{targetAngles[0]}|{targetAngles[1]}|{targetAngles[2]}|{targetAngles[3]}|{targetAngles[4]}|{targetAngles[5]}|60|"
    f"{targetDistances[0]}|{targetDistances[1]}|{targetDistances[2]}|{targetDistances[3]}|{targetDistances[4]}|{targetDistances[5]}|{targetDistances[6]}"
)

In [684]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-05-12 18:10:46,455 | INFO     |:  | Number of observations per grid cell: 18
2025-05-12 18:10:46,458 | INFO     |:  | Precaching Views...
2025-05-12 18:10:48,942 | INFO     |:  | Precaching Time: 2.484 secs
2025-05-12 18:10:48,944 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-12 18:10:48,945 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [685]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_odom(current_odom[0], current_odom[1])

2025-05-12 18:10:49,975 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-05-12 18:10:49,977 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2025-05-12 18:11:13,421 | INFO     |: Disconnected from C0:81:80:25:12:64


CancelledError: 

In [89]:
cmdr.plot_gt(0,0.9914) # 0, 3 = 0, 0.9144
# cmdr.plot_gt(1.524,0.9914) # 5, 3 = 1.524, 0.9144
# cmdr.plot_gt(1.524,-0.9914) # 5, -3 = 1.524, -0.9144
# cmdr.plot_gt(-0.9144,-0.6096) # -3,-2 = -0.9144, -0.6096

2025-04-23 16:08:44,550 | INFO     |: Disconnected from C0:81:80:25:12:64
